In [11]:
!pip install pandas geopy

In [12]:
!pip install openpyxl

In [13]:
#Dependencies
import pandas as pd
import os
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
from geopy.exc import GeocoderInsufficientPrivileges
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Initialize geolocator
geolocator = Nominatim(user_agent="Noah Bazis Geocoding for SSP Analysis")

def get_lat_lon(row):
    address = row['Address']
    for _ in range(3):  # Try 3 times
        try:
            location = geolocator.geocode(address, timeout=10)
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except GeocoderTimedOut:
            time.sleep(2)  # Sleep for 2 seconds before retrying
            continue
        except GeocoderInsufficientPrivileges:
            time.sleep(2)
            continue
    return None, None

In [15]:
#Set file path
file_path = '/content/drive/MyDrive/IJ Data Analysis/SSP/SSP_APPS_RAW.csv'

In [16]:
# Read the CSV into a DataFrame
df = pd.read_csv(file_path,  encoding='ISO-8859-1')
df

,Screening. Consider?,Business Name,Website of Business (if any),What neighborhood is your business located in (or home if your business does not have a physical location)?,Business Address (Street Address),Business Address (Address Line 2),Business Address (ZIP / Postal Code),What is your business connection to the South Side?,Describe in detail what your business or product is. What does it do?,Describe your journey as an entrepreneur and tell us the history of your business.,...,Aleksei Score,Beth's Comments,Beth Score,Cathy's Comments,Cathy Score,Elijah's Comments,Elijah Score,Total Score,Semifinalist?,Any follow up or lead?
0,Yes,almghty,www.RBGSKATE.com,Chatham,7627 S. Cottage Grove,NaN,60619,RBG SKATE CENTER is the brainchild of Ernest ...,RBG Skate Center will be a membership business...,I first started a skateboard brand name B-Boar...,...,2.0,"I think this is awesome as a start-up idea, bu...",1 or 5?,1,NaN,I think they need to be clearer history of run...,1.0,#VALUE!,NaN,prospective client? contact to let them know w...
1,Yes,Turner Haus Brewery LLC,turnerhausbrew.com,Bronzeville,78 E 47th St,NaN,60615,Turner Haus Brewery's connection to the South ...,Turner Haus Brewery is a cherished Black-famil...,Turner Haus Brewery's journey mirrors our deep...,...,5.0,lovely story of family roots and gradual expan...,5,NaN,5.0,Solid business all around. Clear mission and e...,5.0,25,NaN,outreach on policy issues -- learn about zonin...
2,Yes,Soul Delivered L3C,www.SoulDelivered.com,NaN,409 E 75th Street,NaN,60619,Covid shutdown many southside businesses. 50% ...,"Soul Delivered, an Illinois L3C low profit, so...",2020 COVID revealed the high expense of the 3r...,...,5.0,cool business with ripple effects on the economy,5,NaN,4.0,Really like how this business is specifcally o...,5.0,23,NaN,NaN
3,Yes,ALMVGHTY CLOTHING,Www.almvghty.com,West Pullman,11137 s Langley ave,NaN,60628,"The founder behind ALMVGHTY CLOTHING, Dondre S...",ALMVGHTY is a Kingdom Building Clothing Compan...,"When I was younger, my mother put me into mode...",...,5.0,distinctive fashion brand with incubator impac...,4,NaN,4.0,Business is very thought out. I like how they ...,5.0,23,NaN,NaN
4,Yes,Crown'd Mane llc.,www.crowndmane.com,Bronzeville,1449 S. Michigan Ave.,NaN,60605,"My name is Khadijah Starks, I am a native of C...",We are an exclusive hair care line that provid...,"As an entrepreneur, I started my journey as a ...",...,5.0,amazing story from homeless to nationwide,5,NaN,4.0,Love the story and vision,5.0,23,NaN,Connect to Beauty Not Barriers. Says she prov...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


In [17]:
#Drop Address (Address Line 2) column
df = df.drop(columns=['Business Address (Address Line 2)'])

# Concatenate and create the "Address" column
df['Address'] = df['Business Address (Street Address)'] + ", " + df['Business Address (ZIP / Postal Code)'].astype(str)

# If you want to drop the original columns after creating the 'Address' column:
df = df.drop(columns=['Business Address (Street Address)', 'Business Address (ZIP / Postal Code)'])

#Show data
df

,Screening. Consider?,Business Name,Website of Business (if any),What neighborhood is your business located in (or home if your business does not have a physical location)?,What is your business connection to the South Side?,Describe in detail what your business or product is. What does it do?,Describe your journey as an entrepreneur and tell us the history of your business.,How has creativity helped you grow your business or adapt in light of challenges? Give us one or two examples.,"What meaningful impact has your business had on your family, community, or industry?",What specific business project would you pitch in front of judges?,...,Beth's Comments,Beth Score,Cathy's Comments,Cathy Score,Elijah's Comments,Elijah Score,Total Score,Semifinalist?,Any follow up or lead?,Address
0,Yes,almghty,www.RBGSKATE.com,Chatham,RBG SKATE CENTER is the brainchild of Ernest ...,RBG Skate Center will be a membership business...,I first started a skateboard brand name B-Boar...,RBG Skate Center open yet. but once we our ope...,RBG Skate Center is not open yet.,RBG would use the prize monies to help open th...,...,"I think this is awesome as a start-up idea, bu...",1 or 5?,1,NaN,I think they need to be clearer history of run...,1.0,#VALUE!,NaN,prospective client? contact to let them know w...,"7627 S. Cottage Grove, 60619"
1,Yes,Turner Haus Brewery LLC,turnerhausbrew.com,Bronzeville,Turner Haus Brewery's connection to the South ...,Turner Haus Brewery is a cherished Black-famil...,Turner Haus Brewery's journey mirrors our deep...,Turner Haus Brewery's journey has been fueled ...,Turner HÃ¤us Brewery has made significant impa...,Turner Haus Brewery has a pitch for South Side...,...,lovely story of family roots and gradual expan...,5,NaN,5.0,Solid business all around. Clear mission and e...,5.0,25,NaN,outreach on policy issues -- learn about zonin...,"78 E 47th St, 60615"
2,Yes,Soul Delivered L3C,www.SoulDelivered.com,NaN,Covid shutdown many southside businesses. 50% ...,"Soul Delivered, an Illinois L3C low profit, so...",2020 COVID revealed the high expense of the 3r...,"We are action oriented, try things, evaluate, ...",Every modern business has to have a digital pr...,We need marketing to generate demand for deliv...,...,cool business with ripple effects on the economy,5,NaN,4.0,Really like how this business is specifcally o...,5.0,23,NaN,NaN,"409 E 75th Street, 60619"
3,Yes,ALMVGHTY CLOTHING,Www.almvghty.com,West Pullman,"The founder behind ALMVGHTY CLOTHING, Dondre S...",ALMVGHTY is a Kingdom Building Clothing Compan...,"When I was younger, my mother put me into mode...","On March 20th, 2020 a significant milestone wa...","Undoubtedly, one of the most profound impacts ...",I'm excited to present a transformative projec...,...,distinctive fashion brand with incubator impac...,4,NaN,4.0,Business is very thought out. I like how they ...,5.0,23,NaN,NaN,"11137 s Langley ave, 60628"
4,Yes,Crown'd Mane llc.,www.crowndmane.com,Bronzeville,"My name is Khadijah Starks, I am a native of C...",We are an exclusive hair care line that provid...,"As an entrepreneur, I started my journey as a ...",One of the biggest points of creativity within...,Crown'd Mane has had an impact on my immediate...,Now that Crown'd Mane is past the beginning st...,...,amazing story from homeless to nationwide,5,NaN,4.0,Love the story and vision,5.0,23,NaN,Connect to Beauty Not Barriers. Says she prov...,"1449 S. Michigan Ave., 60605"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


In [18]:
#NOTE:Code works, but apt/suite numbers in addresses results in a null - must remove for proper calculation
#Made edits in Excel; try regex when time permits
df["Address"].tolist()

['7627 S. Cottage Grove, 60619',
 '78 E 47th St, 60615',
 '409 E 75th Street, 60619',
 '11137 s Langley ave, 60628',
 '1449 S. Michigan  Ave., 60605',
 '6127 S. University Ave, 60637',
 '4072 South Lake Park Avenue, 60653',
 '2423 E. 75th St, 60649',
 '1546 W 63rd St, 60636',
 '841 W 103rd Street, 60643',
 '6507 S Kimbark Ave, 60637',
 '1307 W. 18th Street, 60608',
 '9828 S Aberdeen St, 60643',
 '359 E 79th Street, 60619',
 '330 East 51st Street, 60615',
 '3011 W 19th St., 60623',
 '4710 S. ELLIS Ave, 60615',
 '10735 S. Western Ave., 60643',
 '621 E 67th Street, 60637',
 '8340 s. Stony island ave, 60617',
 '2930 S. Michigan Ave, 60616',
 '1647 East 71st Street, 60649',
 '330 E. 51st St., 60619',
 '536 E 95th St, 60619',
 '6848 S Ashland Ave, 60636',
 '1017 S. Western Avenue, 60612',
 '4553 S State Street, 60609',
 '408 W 71st Street, 60621',
 '8514 S. Yates Blvd, 60617',
 '6024 S Princeton ave, 60621',
 '13257 S. Baltimore Ave, 60633',
 '700 E 61st St, 60637',
 '329 West 18th Street , 

In [19]:
# Get the latitude and longitude for each row
df['Lat'], df['Lon'] = zip(*df.apply(get_lat_lon, axis=1))

# Save the DataFrame with the new columns back to a CSV
df.to_csv('SSP_APPS_LAT_LON.csv', index=False)

print("Geocoding completed and saved to 'SSP_APPS_LAT_LON.csv'.")

Geocoding completed and saved to 'SSP_APPS_LAT_LON.csv'.


In [20]:
#Save output to Google Drive
import shutil
shutil.copy('SSP_APPS_LAT_LON.csv', '/content/drive/MyDrive/IJ Data Analysis/SSP/')


'/content/drive/MyDrive/IJ Data Analysis/SSP/SSP_APPS_LAT_LON.csv'